#### Forbes 2000

In [ ]:
import re
import time
import xlwt
import random

import numpy as np
import pandas as pd
import requests
from lxml import etree
from bs4 import BeautifulSoup

# 使用socks代理
proxies = {
    'http': 'socks5h://127.0.0.1:4781', 
    'https': 'socks5h://127.0.0.1:4781'
}
# 设置头文件
headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    'Sec-Fetch-Site': 'same-origin',
    'Cookie':'__Secure-3PSID=aAh2vZj3NYq7s8-z7KtaXD9altN5PbXXuGuvmGgtwB0-qMSSDGcRBZiQ94nBMOHmXj6q2w.; __Secure-3PAPISID=yLLrrGsX1O1Lx0a8/A_LX3wF37TDIrDdop; __Secure-3PSIDTS=sidts-CjEBSAxbGfC_IvmfEV_qELFBLxdBNFr_xprSxhljrBj5e3NDVpCo0BmpcWyAa9gBZBNIEAA; NID=511=XtwrWjOckhXH8IwYDV1f8wUVGabbibpNi47NJvUXMcGvlLKDUF2eV3pg0CI5b6DVlnPDgSiEAEXRUtburci1HiCRraSoIHOq83znzy8nH1UFY52oKmz5M0V48LWGiY7LkTwKDeMC3JWsO86JqnTtEadAbUTjKPEsdzumKvf6O4GYqA4meRG6Zf607rc087sNZ3uJyqrbjth5BcgF8rn4teD7zomX2OlawA2-GPQbIeESj4OMXwXrJVdhn0Y4lujodiIdssvWHEn6NR_ak4BF0YoDmtnxA8AZAiLILi4; 1P_JAR=2023-08-23-06; __Secure-3PSIDCC=APoG2W92Px9j1VZ7fB4momL0cvGCrO27opI5UdfaNEYNc5kkVeKfQ5RBjIbxyx0ZSl97QiOQjw'
}

def get_company(url):
    # 获取企业数据
    html = requests.get(url, headers=headers, proxies=proxies).text    
    data = etree.HTML(html)
    rows = data.xpath('//div[@class="table-row-group"]/a')
    print(len(rows))
    
    data = pd.DataFrame(columns=['RANK', 'NAME', 'COUNTRY', 'SALES', 'PROFIT', 'ASSETS', 'MARKET', 'LINK'])
    for row in rows:
        # 获取企业数据及页面链接
        RANK = ''.join(row.xpath('./div[1]/text()')).strip()[:-1]
        NAME = ''.join(row.xpath('./div[2]/text()')).strip()
        COUNTRY = ''.join(row.xpath('./div[3]/text()')).strip()
        SALES = ''.join(row.xpath('./div[4]/text()')).strip()
        PROFIT = ''.join(row.xpath('./div[5]/text()')).strip()
        ASSETS = ''.join(row.xpath('./div[6]/text()')).strip()
        MARKET = ''.join(row.xpath('./div[7]/text()')).strip()
        if len(row.xpath('./@href'))>0:
            LINK = row.xpath('./@href')[0]
        else:
            LINK = ''
        value = [RANK, NAME, COUNTRY, SALES, PROFIT, ASSETS, MARKET, LINK]
        # 打印结果
        print(value)
        
        data.loc[len(data.index)] = value
    
    return data

def get_location(data):
    data['INDUSTRY'], data['HEADQUARTERS']= None, None
    for i in range(len(data)):
        # 获取企业页面URL
        url = data.iloc[i, 7]
        try:
            html = requests.get(url, headers=headers, proxies=proxies, timeout=30).text
            INDUSTRY = re.findall(
                '<dt class="profile-stats__title">Industry</dt><dd class="profile-stats__text"><span class="profile-stats__text">(.*?)</span>',
                html, re.S)[0]
            HEADQUARTERS = re.findall(
                '<dt class="profile-stats__title">Headquarters</dt><dd class="profile-stats__text"><span class="profile-stats__text">(.*?)</span>',
                html, re.S)[0]
            print(url, INDUSTRY, HEADQUARTERS)
        except Exception as e:
            print(e)
            INDUSTRY = ""
            HEADQUARTERS = ""
        time.sleep(random.randint(1, 3))
            
        data.iloc[i, 8], data.iloc[i, 9] = INDUSTRY, HEADQUARTERS
    
    data.to_excel('Forbes 2000.xlsx', index=False)

In [ ]:
url = 'https://www.forbes.com/lists/global2000/?sh=30d6e68a5ac0'
data = get_company(url)
result = get_location(data)